# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/vl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn import naive_bayes, svm




In [10]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df =  pd.read_sql_query ( "SELECT * FROM LabeledMessages", engine)

#Transform values in binary
df['related'] = df['related'].astype('str').str.replace('2', '1')
df['related'] = df['related'].astype('int')

X = df["message"]
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
df.head()

X shape: (26216,)
y shape: (26216, 36)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Transform values in binary

In [4]:
message_content_count = df.drop(['id', 'message', 'original', 'genre'], axis=1).sum().index

message_content_count

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    norm_text = text.lower()
    tokens = word_tokenize(norm_text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def randon_forest_pipeline():
    pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),

        ('clf', RandomForestClassifier())
    ])
    
    multi_target_linear = MultiOutputClassifier(pipeline) 
    
    return multi_target_linear

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

randon_forest_model = randon_forest_pipeline()
randon_forest_model.fit(X_train, y_train)

y_pred = randon_forest_model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def display_results(y_test, y_pred):
    
    category_accuracy = (y_pred == y_test).mean()
    accuracy = category_accuracy.mean()
    
    print("Total Accuracy: \n", accuracy)
    print("Label Accuracy: \n", category_accuracy)
    
    for column in range(y_test.shape[1]):
        print("Column: ",y.columns[column])
        print(classification_report(y_test.to_numpy()[:,column], y_pred[:,column]))
        print("----------------------------------------------------------*")



In [11]:
display_results(y_test, y_pred)

Total Accuracy: 
 0.9463389617875428
Label Accuracy: 
 related                   0.809429
request                   0.897162
offer                     0.996033
aid_related               0.767318
medical_help              0.923253
medical_products          0.953921
search_and_rescue         0.974672
security                  0.981996
military                  0.966890
child_alone               1.000000
water                     0.950107
food                      0.924626
shelter                   0.931187
clothing                  0.985200
money                     0.978944
missing_people            0.989319
refugees                  0.967806
death                     0.960940
other_aid                 0.868325
infrastructure_related    0.934849
transport                 0.956515
buildings                 0.954074
electricity               0.979860
tools                     0.994202
hospitals                 0.989167
shops                     0.995575
aid_centers               0.986421


/home/vl/Courses/Udacity/dataScience/project1/CRISP-DM/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vl/Courses/Udacity/dataScience/project1/CRISP-DM/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vl/Courses/Udacity/dataScience/project1/CRISP-DM/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
def model_pipeline_grid():
    pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),

        ('clf', RandomForestClassifier())
    ])
    
    parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        #'tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        #'clf__min_samples_split': [2, 3, 4],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    multi_target_linear = MultiOutputClassifier(cv) 
    
    return multi_target_linear



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model_grid = model_pipeline_grid()
model_grid.fit(X_train, y_train)

y_pred = model_grid.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(model_grid.best_params_)
print(model_grid.score(X_test,np.array(y_test)))

display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def simpler_model_pipeline():
    pipeline = Pipeline([
        
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('lr_multi', MultiOutputClassifier(
            RandomForestClassifier())
        
        )])
    
    return pipeline


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

smp_model = simpler_model_pipeline()
smp_model.fit(X_train, y_train)

y_pred = smp_model.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

In [ ]:
print(f"model X score: {randon_forest_model.score(X_test,np.array(y_test))}")
print(f"model X score: {model_grid.score(X_test,np.array(y_test))}")
print(f"model X score: {smp_model.score(X_test,np.array(y_test))}")

In [ ]:
def best_model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        
        ('lr_multi', MultiOutputClassifier(
                MultinomialNB()
        ))])
    
    return pipeline

In [ ]:
best_model = best_model_pipeline()
best_model.fit(X_train, y_train)

print(best_model.score(X_test,np.array(y_test)))
y_pred = best_model.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle.dump(best_model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.